In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt
import keras_cv
from keras_cv.models import StableDiffusion
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
import tensorflow.keras as keras
import time


SAVE_PATH = "/users/skoka/Documents/Final_Paper_ML"


Stable_diffusion = StableDiffusion(img_height=512, img_width=512)
decoder = Stable_diffusion.decoder

diffusion_model = DiffusionModel(img_width = 512, img_height = 512, max_text_length=0)

import math
def get_timestep_embedding(timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

# we are going to need to modify this code a lot 
def generate_pattern(layer_index, filter_index, size=64):
    # Build a model that outputs the activation
    # of the nth filter of the layer considered.
    layer_output = diffusion_model.layers[layer_index].output
    # Isolate the output 
    new_model = tf.keras.models.Model(inputs=diffusion_model.inputs, outputs=layer_output)
    
    # We start from a gray image with some uniform noise
    input_img_data = np.random.random((1, size, size, 4)) * 20 + 128.
    I = tf.Variable(input_img_data, name='image_var', dtype = 'float64')
    #I = preprocess_input(I_start) # only process once
    # Run gradient ascent for 40 steps
    eta = 5
    for i in range(100):
        start = time.time()
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            time_embedding = get_timestep_embedding(i, 1)
            # create a tensor with the following size of all zeros: (None, 0, 768)
            word_embedding = tf.zeros((1, 0, 768))
            tape.watch(I)
            # get variable to maximize
            model_vals = new_model((I, time_embedding, word_embedding))
            loss = tf.reduce_mean(model_vals[:, :, :, filter_index])

        # Compute the gradient of the input picture w.r.t. this loss
        # add this operation input to maximize
        grad_fn = tape.gradient(loss, I)
        # Normalization trick: we normalize the gradient
        grad_fn /= (tf.sqrt(tf.reduce_mean(tf.square(grad_fn))) + 1e-5) # mean L2 norm
        I = I + (grad_fn * eta) # one iteration of maximizing
        end = time.time()
        print("Iteration: {}, Loss: {}, Time: {}".format(i, loss, end-start))
    # decode the resulting input image
    I = decoder(I)
    # return the numpy matrix so we can visualize 
    img = I.numpy()
    return img

2023-05-09 21:43:18.465027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 21:43:20.756045: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.7.0-vbhdtgc7dl4kpo4auyswsh6w3udcnf5x/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-05-09 21:43:20.756194: W tensorflow/compiler/xla/stream_execut

2.11.0
You do not have pycocotools installed, so KerasCV pycoco metrics are not available. Please run `pip install pycocotools`.
You do not have pyococotools installed, so the `PyCOCOCallback` API is not available.
You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


/users/skoka/.venv/MLEnv3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
cnn_layers_in_order = []

for layer in range(len(diffusion_model.layers)):
    curr_layer = diffusion_model.layers[layer]
    if len(curr_layer.get_weights()) != 0:
        for sublayer in range(len(curr_layer.get_weights())):
            shape = curr_layer.get_weights()[sublayer].shape
            if len(shape) == 4 and shape[0] == 3 and shape[1] == 3:
                cnn_layers_in_order.append((layer, sublayer))

def get_cnn_layer(index):
    return diffusion_model.layers[cnn_layers_in_order[index][0]].get_weights()[cnn_layers_in_order[index][1]]

In [4]:
cnn_layers_in_order

[(4, 0),
 (6, 2),
 (6, 8),
 (9, 2),
 (9, 8),
 (11, 0),
 (12, 2),
 (12, 8),
 (14, 2),
 (14, 8),
 (16, 0),
 (17, 2),
 (17, 8),
 (19, 2),
 (19, 8),
 (21, 0),
 (22, 2),
 (22, 8),
 (23, 2),
 (23, 8),
 (24, 2),
 (24, 8),
 (26, 2),
 (26, 8),
 (28, 2),
 (28, 8),
 (30, 2),
 (30, 8),
 (32, 2),
 (32, 8),
 (33, 0),
 (35, 2),
 (35, 8),
 (38, 2),
 (38, 8),
 (41, 2),
 (41, 8),
 (43, 0),
 (45, 2),
 (45, 8),
 (48, 2),
 (48, 8),
 (51, 2),
 (51, 8),
 (53, 0),
 (55, 2),
 (55, 8),
 (58, 2),
 (58, 8),
 (61, 2),
 (61, 8),
 (65, 0)]

In [3]:
for i in range(len(cnn_layers_in_order)):
    print("Layer: {}, Shape: {}".format(i, get_cnn_layer(i).shape))

Layer: 0, Shape: (3, 3, 4, 320)
Layer: 1, Shape: (3, 3, 320, 320)
Layer: 2, Shape: (3, 3, 320, 320)
Layer: 3, Shape: (3, 3, 320, 320)
Layer: 4, Shape: (3, 3, 320, 320)
Layer: 5, Shape: (3, 3, 320, 320)
Layer: 6, Shape: (3, 3, 320, 640)
Layer: 7, Shape: (3, 3, 640, 640)
Layer: 8, Shape: (3, 3, 640, 640)
Layer: 9, Shape: (3, 3, 640, 640)
Layer: 10, Shape: (3, 3, 640, 640)
Layer: 11, Shape: (3, 3, 640, 1280)
Layer: 12, Shape: (3, 3, 1280, 1280)
Layer: 13, Shape: (3, 3, 1280, 1280)
Layer: 14, Shape: (3, 3, 1280, 1280)
Layer: 15, Shape: (3, 3, 1280, 1280)
Layer: 16, Shape: (3, 3, 1280, 1280)
Layer: 17, Shape: (3, 3, 1280, 1280)
Layer: 18, Shape: (3, 3, 1280, 1280)
Layer: 19, Shape: (3, 3, 1280, 1280)
Layer: 20, Shape: (3, 3, 1280, 1280)
Layer: 21, Shape: (3, 3, 1280, 1280)
Layer: 22, Shape: (3, 3, 1280, 1280)
Layer: 23, Shape: (3, 3, 1280, 1280)
Layer: 24, Shape: (3, 3, 2560, 1280)
Layer: 25, Shape: (3, 3, 1280, 1280)
Layer: 26, Shape: (3, 3, 2560, 1280)
Layer: 27, Shape: (3, 3, 1280, 1280)

In [37]:
def l2_norm(layer_weights):
    num_filters = layer_weights.shape[3]
    norms_list = []
    prev_filters_list = []
    curr_filters_list = []
    for filter in range(num_filters):
        l2_norms = np.linalg.norm(layer_weights[:,:,:,filter], axis = (0,1))
        # get max l2 norm
        norms_list.append(np.max(l2_norms))
        curr_filters_list.append(filter)
        prev_filters_list.append(np.argmax(l2_norms))
    norms_list = np.array(norms_list)
    prev_filters_list = np.array(prev_filters_list)
    index = np.argmax(norms_list)
    return norms_list.max(), prev_filters_list[index], curr_filters_list[index]

def get_l2_norms_of_single_channel_filter(layer_index, channel_index):
    layer_weights = get_cnn_layer(layer_index)
    l2_norms = np.linalg.norm(layer_weights[:,:,:,channel_index], axis = (0,1))
    top_5_indices = np.argsort(l2_norms)[-15:]
    top_5_norms = l2_norms[top_5_indices]
    return top_5_norms, top_5_indices
    

In [38]:
top_indices = []
top_norms = []
for i in range(4):
    norms, indices = get_l2_norms_of_single_channel_filter(51, i)

    new_indices = []
    for index in indices:
        new_indices.append((index, i))
    # add the norms to the list of top norms
    top_norms += list(norms)
    top_indices += list(new_indices)

# sort the norms and indices by the values of the norms
top_norms = np.array(top_norms)
top_indices = np.array(top_indices)
sorted_indices = np.argsort(top_norms)[::-1]
top_norms = top_norms[sorted_indices]
top_indices = top_indices[sorted_indices]

top_indices


array([[303,   3],
       [301,   3],
       [300,   1],
       [293,   3],
       [ 46,   2],
       [ 42,   1],
       [309,   3],
       [305,   0],
       [298,   2],
       [309,   2],
       [306,   2],
       [300,   2],
       [ 41,   3],
       [224,   3],
       [ 48,   3],
       [308,   2],
       [302,   0],
       [308,   3],
       [ 90,   3],
       [113,   3],
       [ 45,   1],
       [112,   3],
       [291,   2],
       [116,   1],
       [298,   3],
       [176,   1],
       [236,   1],
       [119,   2],
       [ 36,   2],
       [ 45,   3],
       [118,   2],
       [292,   3],
       [226,   2],
       [ 48,   2],
       [297,   0],
       [ 41,   0],
       [295,   2],
       [228,   3],
       [227,   1],
       [304,   0],
       [111,   0],
       [303,   0],
       [ 43,   1],
       [222,   1],
       [  4,   1],
       [ 41,   2],
       [233,   2],
       [190,   0],
       [225,   1],
       [ 33,   1],
       [205,   1],
       [309,   0],
       [296,

In [39]:
def get_circuit(layer_index, prev_layer_filter, curr_layer_channel):
    filters_to_visualize = []
    filters_to_visualize.append((layer_index, curr_layer_channel))
    layer_index -= 1
    while layer_index > 0:
        max_l2_norms, filter_numbers = get_l2_norms_of_single_channel_filter(layer_index, prev_layer_filter)
        # print(get_cnn_layer(layer_index).shape, prev_layer_filter)
        filters_to_visualize.append((layer_index, prev_layer_filter))
        prev_layer_filter = filter_numbers[-1]
        i = -2
        while prev_layer_filter > get_cnn_layer(layer_index-1).shape[3]:
            print(layer_index, prev_layer_filter, get_cnn_layer(layer_index).shape, get_cnn_layer(layer_index-1).shape)
            prev_layer_filter = filter_numbers[i]
            i -= 1
        layer_index -= 1
    print(filters_to_visualize)
    return filters_to_visualize
    
circuit = get_circuit(len(cnn_layers_in_order)-1, 46, 2)

38 1585 (3, 3, 1920, 640) (3, 3, 1280, 1280)
31 2363 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2112 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 1731 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2170 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 1877 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2040 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2413 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2184 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 1356 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 1872 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2080 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 1536 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
31 2559 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
26 2363 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
26 2242 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
26 1300 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
26 1408 (3, 3, 2560, 1280) (3, 3, 1280, 1280)
[(51, 2), (50, 46), (49, 25), (48, 290), (47, 221), (46, 16), (45, 165), (44, 397), (43, 421), (42, 629), (41, 170), (40, 476), (39, 49), (38, 574), (37, 603), (36, 366), (3

In [40]:
i = 0
prev_layer_index = 0
visualize = []
while i < len(circuit):
    layer_index = circuit[i][0]
    filter_index = circuit[i][1]
    cnn_layers_in_order[layer_index][0]
    if cnn_layers_in_order[layer_index][0] == cnn_layers_in_order[prev_layer_index][0]:
        i += 1
        continue
    prev_layer_index = layer_index
    visualize.append((cnn_layers_in_order[layer_index][0], circuit[i][1]))

In [41]:
visualize

[(65, 2),
 (61, 46),
 (58, 290),
 (55, 16),
 (53, 397),
 (51, 421),
 (48, 170),
 (45, 49),
 (43, 603),
 (41, 366),
 (38, 1143),
 (35, 511),
 (33, 617),
 (32, 879),
 (30, 668),
 (28, 134),
 (26, 20),
 (24, 5),
 (23, 367),
 (22, 245),
 (21, 1132),
 (19, 22),
 (17, 21),
 (16, 343),
 (14, 632),
 (12, 392),
 (11, 237),
 (9, 187),
 (6, 177)]

In [42]:
# save visualize to a file called args.txt
# it should be formatted by having each tuple on a new line with no commas or parentheses

with open(SAVE_PATH + '/args.txt', 'w') as f:
    for item in visualize:
        f.write(str(item[0]) + ' ' + str(item[1]) + '\n')